In [91]:
from arff_parser import *
import numpy as np
import sys
import math

In [145]:
class Question:
    def __init__(self, arffdata, feature_num, value):
        #feature is the column number in the data set, nominal=1: feature is nominal
        self.feature = arffdata.attributes[feature_num].name
        self.value = value
        self.type = arffdata.all_attributes[feature_num].type
    
    def is_numeric(self):
        if arffdata.all_attributes[feature_num].type == "real":
            return 1
        if arffdata.all_attributes[feature_num].type == "nominal":
            return 0
        
    def __repr__(self):
        if arffdata.all_attributes[feature_num].type == "nominal":
            return "%s == %s ?" % (self.feature, self.value)
        if arffdata.all_attributes[feature_num].type == "real":
            return "%s > %s ?" % (self.feature, self.value)

In [93]:
def get_feature_data(column_num, data):
    column = []
    for row in data:
        column.append(row[column_num])
    return column

In [94]:
#given the arff data and feature number, return a list of candidate splits
#arffdata carries feature info and data carries actual data
def determine_candidate_split(arffdata, data, feature_num):
    feature_value_list = get_feature_data(feature_num, data)
    
    if arffdata.all_attributes[feature_num].type == "real":
        feature_value_list = sorted(feature_value_list)
        candidates = []
        for i in range(len(feature_value_list)-1):        
            candidates.append((feature_value_list[i]+feature_value_list[i+1])/2)
        return sorted(list(set(candidates)))
    
    if arffdata.all_attributes[feature_num].type == "nominal":
        candidates = arffdata.all_attributes[feature_num].attribute_list
        return candidates

In [95]:
def split_data(data, arffdata, feature_num, threshold):
    attributes = arffdata.all_attributes[feature_num].attribute_list
    if arffdata.all_attributes[feature_num].type == "real":
        nominal = 0
    if arffdata.all_attributes[feature_num].type == "nominal":
        nominal = 1
    splited_data = []
    
    if nominal == 1:
        for i in range(len(attributes)):
            split = []
            for j in data:
                if j[feature_num] == attributes[i]:
                    split.append(j)
            splited_data.append(split)
    
    if nominal == 0:
        left = []
        right = []
        for i in data:
            if i[feature_num] > threshold:
                left.append(i)
            else:
                right.append(i)
        splited_data.append(left)
        splited_data.append(right)
        
    return splited_data

In [96]:
def label_entropy(data, arffdata):
    cnt = 0
    for i in data:
        if arffdata.label.attribute_list[0] == i[-1]:
            cnt += 1
    p = cnt/len(data)
    return -p*math.log2(p)-(1-p)*math.log2(1-p)

In [97]:
def Entropy(p):
    return -p*math.log2(p)

In [169]:
def real_feature_entropy(data, arffdata, feature_num, threshold):
    cnt_greater = 0
    cnt_pos_greater = 0
    cnt_pos_less = 0
    for i in data:
        if i[feature_num] > threshold:
            cnt_greater += 1
        if i[-1] == arffdata.label.attribute_list[0] == i[-1] and i[feature_num] > threshold:
            cnt_pos_greater += 1
        if i[-1] == arffdata.label.attribute_list[0] == i[-1] and i[feature_num] <= threshold:
            cnt_pos_less += 1
    p_greater = cnt_greater/len(data)
    if cnt_greater == 0:
        p_cnt_pos_greater = 0
    else:
        p_cnt_pos_greater = cnt_pos_greater/cnt_greater
    p_cnt_pos_less = cnt_pos_less/(len(data)-cnt_greater)
    if p_cnt_pos_greater != 0:
        E1 = Entropy(p_cnt_pos_greater)
    else:
        E1 = 0
    if 1-p_cnt_pos_greater != 0:
        E2 = Entropy(1-p_cnt_pos_greater)
    else:
        E2 = 0
    if p_cnt_pos_less != 0:
        E3 = Entropy(p_cnt_pos_less)
    else:
        E3 = 0
    if 1-p_cnt_pos_less:
        E4 = Entropy(1-p_cnt_pos_less)
    else:
        E4 = 0
    entropy = p_greater*(E1+E2) + (1-p_greater)*(E3+E4)
    return entropy

In [170]:
a = arff_data("credit_train.arff")
real_feature_entropy(a.data, a, 6, 5)

0.699136348426275

In [120]:
label_entropy(a.data, a)

0.8653409924402704

In [214]:
def nominal_feature_entropy(data, arffdata, feature_num):
    entropy = 0
    for i in arffdata.all_attributes[feature_num].attribute_list:
        cnt = 0
        cnt_pos = 0   
        for row in data:
            if row[feature_num] == i:
                cnt += 1
            if row[feature_num] == i and row[-1] == arffdata.label.attribute_list[0]:
                cnt_pos += 1
        if cnt == 0:
            p_cnt_pos = 0
        else:
            p_cnt_pos = cnt_pos/cnt
        if p_cnt_pos != 0:
            E1 = Entropy(p_cnt_pos)
        else:
            E1 = 0
        if 1- p_cnt_pos != 0:
            E2 = Entropy(1-p_cnt_pos)
        else:
            E2 = 0
        entropy += (cnt/len(data))*(E1+E2)
    return entropy

In [215]:
info_gain(a.data, a, 0, 0)

0.0005402573000333755

In [216]:
def entropy(data, arffdata, feature_num, threshold):
    if arffdata.all_attributes[feature_num].type == "real":
        return real_feature_entropy(data, arffdata, feature_num, threshold)
    if arffdata.all_attributes[feature_num].type == "nominal":
        return nominal_feature_entropy(data, arffdata, feature_num)

In [217]:
def info_gain(data, arffdata, feature_num, threshold):
    return label_entropy(data, arffdata)-entropy(data, arffdata, feature_num, threshold)

In [218]:
def find_best_numeric_candidate(data, arffdata, feature_num):
    candidates = determine_candidate_split(arffdata, data, feature_num)
    best_candidate = candidates[0]
    best_info_gain = info_gain(data, arffdata, feature_num, candidates[0])
    
    for i in range(1, len(candidates)):
        if info_gain(data, arffdata, feature_num, candidates[i]) > best_info_gain:
            best_info_gain = info_gain(data, arffdata, feature_num, candidates[i])
            best_candidate = candidates[i]
    return best_candidate, best_info_gain

In [219]:
find_best_numeric_candidate(a.data, a, 7)

(492.0, 0.0734723112044664)

In [220]:
info_gain(a.data, a, 3, 3)

0.01999003567874491

In [221]:
determine_candidate_split(a, a.data, 6)[19]

10.0

In [222]:
class Node:
    def __init__(self, question, data):
        self.question = question
        self.data = data
        self.children = []
        
    def add_children(self, child):
        if isinstance(child, Node):
            self.children.append(child)
        else:
            print("Error: Children must be of Node type")
    def __repr__(self):
        if self.question.is_numeric() == 1:
            return "%s == %s ?" % (self.question.feature, self.question.value)
        if self.question.is_numeric() == 0:
            return "%s > %s ?" % (self.question.feature, self.question.value)

In [223]:
class Leaf:
    def __init__(self, data):
        self.data = data

In [224]:
def if_same_class(data):
    if len(list(set(get_feature_data(-1, data)))) == 1:
        return 1
    else:
        return 0

In [225]:
if_same_class(a.data)

0

In [234]:
def find_best_split(data, arffdata):
    best_split = 0
    best_info_gain = -10000000
    
    for i in range(len(arffdata.attributes)):
        if arffdata.attributes[i].type == "real":
            numeric, _ = find_best_numeric_candidate(data, arffdata, i)
            gain = info_gain(data, arffdata, i, numeric)
            if gain > best_info_gain:
                best_info_gain = gain
                best_split = i
        if arffdata.attributes[i].type == "nominal":
            gain = info_gain(data, arffdata, i, 0)
            if gain > best_info_gain:
                best_info_gain = gain
                best_split = i
        print(gain)
    return best_split

In [235]:
find_best_split(a.data, a)

0.0005402573000333755
0.02993379281204922
0.07377005439269024
0.01999003567874491
0.01999003567874491
0.14837507956162044
0.1922114012110726
0.0734723112044664


6

In [228]:
info_gain(a.data, a, 0, 0)

0.0005402573000333755

In [229]:
find_best_numeric_candidate(a.data, a, 6)

(2.0, 0.1922114012110726)

In [ ]:
def build_tree(sub_data, arffdata, m):
    if len(sub_data) <= m or if_same_class(sub_data) == 1:
        return Leaf(sub_data)
    else:
        